# MD Simulation Free-State Analysis

### 🔍 Caricamento delle dipendenze necessarie
Assicurarsi di aver scaricato tutte le dipendenze necessarie. Non andare avanti se dopo aver eseguito questa prima parte escono degli errori

In [15]:
# ✅ BLOCCO 1: Caricamento dipendenze
import os
import MDAnalysis as mda
from IPython.display import display, clear_output
import ipywidgets as widgets
from tqdm.notebook import tqdm
from ipywidgets import Text, Button, VBox, Output, Layout
from IPython.display import display
import os
from datetime import datetime
print("✅ Dipendenze caricate correttamente")

# --- Selezione e creazione directory output ---
output_save_dir = Output()

save_dir_widget = Text(
    value='',
    placeholder='Inserisci il path della directory dove creare la cartella di output',
    description='Path output:',
    layout=Layout(width='70%')
)
create_folder_button = Button(description='Crea cartella output')

def crea_cartella_output(b):
    with output_save_dir:
        output_save_dir.clear_output()
        base_path = save_dir_widget.value.strip()
        if not base_path:
            print("Errore: inserire un path valido.")
            return
        try:
            now = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_folder = os.path.join(base_path, f"MD_Analysis_{now}")
            os.makedirs(output_folder, exist_ok=True)
            print(f"Cartella output creata: {output_folder}")
            global output_dir
            output_dir = output_folder
        except Exception as e:
            print(f"Errore nella creazione della cartella: {e}")

create_folder_button.on_click(crea_cartella_output)

display(VBox([save_dir_widget, create_folder_button, output_save_dir]))


✅ Dipendenze caricate correttamente


### 🔄 Convertitore di file Multimodel.pdb -> trajectory.xtc
Questa cella esegue una conversione dei file multimodello generati da MacroModel in file di traiettoria leggibili da diversi programmi di analisi di dinamica molecolare.

**Ottenere i file multimodello (MULTIMODEL)**
- Dopo aver ottenuto i file *structure.maegz* di dinamica da Macromodel o DESMOND
- Aprire il promp dei comandi di Shodinger Maestro ( Shrodinger command Prompt) ed eseguite il *structconvert.exe*

In [28]:
# ✅ BLOCCO 1: Caricamento dipendenze
import os
import MDAnalysis as mda
from IPython.display import display, clear_output
import ipywidgets as widgets
from tqdm.notebook import tqdm
import time

print("✅ Dipendenze caricate correttamente")

# ✅ BLOCCO 2: Configurazione widget
path_input = widgets.Text(
    placeholder='Incolla il percorso completo del file PDB',
    description='Percorso:',
    layout={'width': '600px'}
)

load_button = widgets.Button(description="📂 Carica Struttura")
convert_button = widgets.Button(description="🔄 Converti in XTC")
progress = widgets.FloatProgress(value=0, min=0, max=1, description='Progresso:', bar_style='')
output = widgets.Output()

display(widgets.VBox([
    path_input,
    widgets.HBox([load_button, convert_button]),
    progress,
    output
]))

# ✅ BLOCCO 3: Funzioni core ottimizzate
def load_pdb_from_path(pdb_path):
    """Carica un PDB multi-model con gestione errori avanzata"""
    try:
        with output:
            clear_output()
            print("⏳ Caricamento in corso...")
        
        # Forza l'aggiornamento dell'output prima del caricamento
        time.sleep(0.1)
        
        if not os.path.exists(pdb_path):
            with output:
                clear_output()
                print(f"❌ File non trovato: {pdb_path}")
            return None
        
        u = mda.Universe(pdb_path)
        
        with output:
            clear_output()
            print(f"✅ Caricamento completato")
            print(f"Percorso: {os.path.abspath(pdb_path)}")
            print(f"Frame: {len(u.trajectory)}")
            print(f"Atomi: {len(u.atoms)}")
            print(f"Residui: {len(u.residues)}")
        
        return u
    
    except Exception as e:
        with output:
            clear_output()
            print(f"❌ Errore durante il caricamento")
            print(f"Tipo errore: {type(e).__name__}")
            print(f"Dettagli: {str(e)}")
        return None

def convert_to_xtc(universe):
    """Conversione in XTC con output ottimizzato"""
    try:
        output_path = os.path.join(os.path.dirname(path_input.value), "trajectory.xtc")
        n_frames = len(universe.trajectory)
        
        with output:
            clear_output()
            print("⏳ Conversione in corso...")
            print(f"Frame totali da processare: {n_frames}")
            print(f"File di output: {output_path}")
        
        progress.max = n_frames
        progress.value = 0
        
        # Conversione con gestione del rate limit
        update_interval = max(1, n_frames // 100)  # Aggiorna progresso ogni 1% o ogni frame
        with mda.Writer(output_path, universe.atoms.n_atoms) as W:
            for i, ts in enumerate(universe.trajectory):
                W.write(universe.atoms)
                if i % update_interval == 0:
                    progress.value = i
                    time.sleep(0.01)  # Riduce il rate dei messaggi
        
        progress.value = n_frames
        progress.bar_style = 'success'
        
        with output:
            clear_output()
            print("✅ Conversione completata con successo")
            print(f"File creato: {os.path.abspath(output_path)}")
            print(f"Dimensioni: {os.path.getsize(output_path)/1024/1024:.2f} MB")
        
        return True
    
    except Exception as e:
        with output:
            clear_output()
            print("❌ Errore durante la conversione")
            print(f"Tipo errore: {type(e).__name__}")
            print(f"Dettagli: {str(e)}")
        return False

# ✅ BLOCCO 4: Gestione eventi ottimizzata
def on_load_click(b):
    global u
    load_button.disabled = True
    try:
        u = load_pdb_from_path(path_input.value.strip())
        convert_button.disabled = not bool(u)
    finally:
        load_button.disabled = False

def on_convert_click(b):
    convert_button.disabled = True
    try:
        if 'u' not in globals() or not u:
            with output:
                clear_output()
                print("❌ Nessuna struttura caricata")
            return
        
        convert_to_xtc(u)
    finally:
        convert_button.disabled = False

load_button.on_click(on_load_click)
convert_button.on_click(on_convert_click)
convert_button.disabled = True


✅ Dipendenze caricate correttamente


### 📥 RMSD and RMSF 
In this section, the molecular simulation trajectory is loaded using MDTraj.

**Required inputs:**
- `.xtc` file containing the trajectory.
- `.pdb` or `.mae` file containing the initial topology file (ligand structure).
Make sure the paths are correct and that the files are in the same directory or accessible from the specified path.

In [26]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import VBox, HBox, Button, Dropdown, FloatText, Output, Label, Layout
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output
import os
from collections import defaultdict

# --- File Choosers ---
topology_chooser = FileChooser()
topology_chooser.title = '<b>Select PDB file (Topology)</b>'
topology_chooser.filter_pattern = ['*.pdb']
topology_chooser.use_title = True

trajectory_chooser = FileChooser()
trajectory_chooser.title = '<b>Select XTC file (Trajectory)</b>'
trajectory_chooser.filter_pattern = ['*.xtc']
trajectory_chooser.use_title = True

output_folder_chooser = FileChooser()
output_folder_chooser.title = '<b>Select Output Folder</b>'
output_folder_chooser.show_only_dirs = True
output_folder_chooser.use_title = True

# --- Widgets ---
timestep_input = FloatText(value=1.0, description='Timestep:', layout=Layout(width='200px'))
time_unit_dropdown = Dropdown(options=['ps', 'ns'], value='ps', description='Unit:')
analysis_dropdown = Dropdown(options=['RMSD', 'RMSF'], value='RMSD', description='Analysis:')
run_button = Button(description='Run Analysis', button_style='info')
plot_output = Output()

# --- GUI Layout ---
analysis_box = VBox([
    Label("Trajectory Analysis (RMSD/RMSF)"),
    topology_chooser,
    trajectory_chooser,
    output_folder_chooser,
    HBox([timestep_input, time_unit_dropdown]),
    analysis_dropdown,
    run_button,
    plot_output
])

def run_analysis(b):
    with plot_output:
        clear_output()
        try:
            pdb_path = topology_chooser.selected
            xtc_path = trajectory_chooser.selected
            output_dir = output_folder_chooser.selected_path

            if not pdb_path or not xtc_path:
                print("❌ Please select both PDB and XTC files.")
                return

            if not output_dir:
                print("❌ Please select an output folder.")
                return

            print("📦 Loading trajectory...")
            traj = md.load(xtc_path, top=pdb_path)
            traj = traj.superpose(traj, 0)

            timestep = timestep_input.value
            time_unit = time_unit_dropdown.value
            time = np.arange(traj.n_frames) * timestep
            if time_unit == 'ns':
                time /= 1000.0

            analysis = analysis_dropdown.value
            if analysis == 'RMSD':
                print("📊 Calculating RMSD...")
                values = md.rmsd(traj, traj, 0) * 10.0
                y_label = 'RMSD (Å)'
                title = 'RMSD'
                filename = os.path.join(output_dir, "RMSD_plot.png")

                plt.figure(figsize=(10, 5), dpi=300)
                plt.plot(time, values, label='RMSD', color='purple')
                plt.xlabel(f'Time ({time_unit})')
                plt.ylabel(y_label)
                plt.title(title)
                plt.legend()
                plt.autoscale()
                plt.tight_layout()
                plt.savefig(filename, dpi=300)
                plt.show()

            else:
                print("📊 Calculating RMSF...")
                mean_positions = np.mean(traj.xyz, axis=0)
                displacements = traj.xyz - mean_positions
                squared_displacements = displacements ** 2
                mean_squared_displacements = np.mean(squared_displacements, axis=0)
                rmsf_atoms = np.sqrt(np.sum(mean_squared_displacements, axis=1)) * 10.0  # Convert nm → Å

                topology = traj.topology
                resid_to_atoms = defaultdict(list)
                for atom in topology.atoms:
                    resid_to_atoms[atom.residue.index].append(atom.index)

                residue_rmsf = []
                for resid, atom_indices in sorted(resid_to_atoms.items()):
                    rmsf_values = rmsf_atoms[atom_indices]
                    rmsf_res = np.mean(rmsf_values)
                    residue_rmsf.append(rmsf_res)

                residue_rmsf = np.array(residue_rmsf)
                y_label = 'RMSF (Å)'
                title = 'RMSF per Residue'
                filename = os.path.join(output_dir, "RMSF_plot.png")

                plt.figure(figsize=(12, 6), dpi=300)
                x = np.arange(1, len(residue_rmsf) + 1)
                plt.plot(x, residue_rmsf, marker='o', color='dodgerblue', label='Residue RMSF')
                plt.xlabel('Residue number')
                plt.ylabel(y_label)
                plt.title(title)
                plt.legend()
                plt.autoscale()
                plt.tight_layout()
                plt.savefig(filename, dpi=300)
                plt.show()

                print(f"✅ Plot saved to: {filename}")

        except Exception as e:
            print(f"❌ Error: {e}")

run_button.on_click(run_analysis)
display(analysis_box)


### 🧪 Glycosidic angle sampling
In this section, the trajectory (file.xtc) of the molecular simulation must be uploaded using MDTraj.
**Required inputs:**
- `.xtc` file containing the trajectory.
- `.pdb` or `.mae` file containing the topology (ligand structure).
Make sure the paths are correct and that the files are in the same directory or accessible from the specified path.
At the end of the process, you can download the graphs in .png, .pdf, or .svg format.

In [2]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, VBox, HBox, Output, Layout
from IPython.display import display, clear_output

# --- Load PDB and XTC file paths ---
pdb_path_widget = widgets.Text(description="PDB Path:", layout=Layout(width='70%'))
xtc_path_widget = widgets.Text(description="XTC Path:", layout=Layout(width='70%'))
load_button = widgets.Button(description="Load Trajectory")

output_load = Output()

def load_trajectory(b):
    with output_load:
        clear_output()
        pdb_path = pdb_path_widget.value.strip()
        xtc_path = xtc_path_widget.value.strip()
        try:
            global traj
            traj = md.load_xtc(xtc_path, top=pdb_path)
            print(f"Trajectory loaded with {traj.n_frames} frames and {traj.n_atoms} atoms.")
            global residues_list
            residues_list = [f"{res.resSeq} {res.name}" for res in traj.topology.residues]
            print(f"Residues found: {len(residues_list)}")
            num_torsions_widget.disabled = False
        except Exception as e:
            print(f"Error loading trajectory: {e}")

load_button.on_click(load_trajectory)

display(VBox([pdb_path_widget, xtc_path_widget, load_button, output_load]))

# --- Torsion selection ---
num_torsions_widget = widgets.BoundedIntText(
    value=1, min=1, max=10, step=1,
    description='Torsions:',
    disabled=True
)

output_torsions = Output()

def update_atoms_for_residue(change, atoms_dd_list):
    val = change['new']
    if not val:
        for dd in atoms_dd_list:
            dd.options = []
            dd.value = None
        return
    resnum, resname = val.split()
    resnum = int(resnum)
    residue = None
    for r in traj.topology.residues:
        if r.resSeq == resnum and r.name == resname:
            residue = r
            break
    if residue is None:
        for dd in atoms_dd_list:
            dd.options = []
            dd.value = None
        return
    opts = [f"{atom.index} - {atom.name}" for atom in residue.atoms]
    for dd in atoms_dd_list:
        dd.options = opts
        dd.value = None

def extract_atom_index(atom_str):
    if not atom_str:
        raise ValueError("No atom selected")
    return int(atom_str.split(' - ')[0])

def create_torsion_block(torsion_idx, torsion_type):
    title = widgets.HTML(value=f"<b>Torsion #{torsion_idx+1} - Angle {torsion_type}</b>")
    resA_dd = widgets.Dropdown(options=residues_list, description='Residue A:', layout=Layout(width='45%'))
    resB_dd = widgets.Dropdown(options=residues_list, description='Residue B:', layout=Layout(width='45%'))

    if torsion_type == 'PHI':
        atoms_resA_dd = [widgets.Dropdown(description=f"Atom A{i+1}:", layout=Layout(width='40%')) for i in range(2)]
        atoms_resB_dd = [widgets.Dropdown(description=f"Atom B{i+1}:", layout=Layout(width='40%')) for i in range(2)]
    else:  # PSI
        atoms_resA_dd = [widgets.Dropdown(description="Atom A1:", layout=Layout(width='40%'))]
        atoms_resB_dd = [widgets.Dropdown(description=f"Atom B{i+1}:", layout=Layout(width='40%')) for i in range(3)]

    resA_dd.observe(lambda change: update_atoms_for_residue(change, atoms_resA_dd), names='value')
    resB_dd.observe(lambda change: update_atoms_for_residue(change, atoms_resB_dd), names='value')

    box_residues = HBox([resA_dd, resB_dd])
    box_atomsA = HBox(atoms_resA_dd)
    box_atomsB = HBox(atoms_resB_dd)

    block = VBox([title, box_residues, box_atomsA, box_atomsB])
    return block, atoms_resA_dd, atoms_resB_dd

def generate_torsion_widgets(change):
    with output_torsions:
        clear_output()
        n = change['new']
        if n < 1:
            print("Enter a valid number > 0")
            return

        torsion_blocks = []

        for i in range(n):
            phi_block, phi_atomsA, phi_atomsB = create_torsion_block(i, 'PHI')
            psi_block, psi_atomsA, psi_atomsB = create_torsion_block(i, 'PSI')

            output_plot = Output()
            fig_container = {"fig": None}

            filename_widget = widgets.Text(
                value=f'torsion_{i+1}',
                description='File name:',
                layout=Layout(width='50%')
            )
            format_widget = widgets.Dropdown(
                options=['png', 'svg', 'pdf'],
                value='png',
                description='Format:'
            )

            def compute_and_plot(b, i=i,
                               phi_atomsA=phi_atomsA, phi_atomsB=phi_atomsB,
                               psi_atomsA=psi_atomsA, psi_atomsB=psi_atomsB,
                               out=output_plot, fig_cont=fig_container):
                with out:
                    clear_output()
                    try:
                        A1 = extract_atom_index(phi_atomsA[0].value)
                        A2 = extract_atom_index(phi_atomsA[1].value)
                        B1 = extract_atom_index(phi_atomsB[0].value)
                        B2 = extract_atom_index(phi_atomsB[1].value)
                        phi_angles = md.compute_dihedrals(traj, [[A1, A2, B1, B2]])

                        A1_psi = extract_atom_index(psi_atomsA[0].value)
                        B1_psi = extract_atom_index(psi_atomsB[0].value)
                        B2_psi = extract_atom_index(psi_atomsB[1].value)
                        B3_psi = extract_atom_index(psi_atomsB[2].value)
                        psi_angles = md.compute_dihedrals(traj, [[A1_psi, B1_psi, B2_psi, B3_psi]])

                        phi_deg = np.degrees(phi_angles).flatten()
                        psi_deg = np.degrees(psi_angles).flatten()

                        fig, ax = plt.subplots(figsize=(6,5), dpi=300)
                        scatter = ax.scatter(phi_deg, psi_deg, c=range(len(phi_deg)), cmap='viridis', s=5)
                        fig.colorbar(scatter, ax=ax, label='Frame index')
                        ax.set_xlabel('Phi (Degrees)')
                        ax.set_ylabel('Psi (Degrees)')
                        ax.set_xlim(-180, 180)
                        ax.set_ylim(-180, 180)
                        ax.set_title(f'Torsion #{i+1} - Phi vs Psi map')
                        ax.grid(False)
                        plt.show()

                        fig_cont["fig"] = fig
                    except Exception as e:
                        print(f"Error during atom selection or computation: {e}")

            btn_calc = widgets.Button(description=f"Compute & Plot Torsion #{i+1}")
            btn_calc.on_click(compute_and_plot)

            def export_figure(b, fig_cont=fig_container,
                             fname_widget=filename_widget,
                             fmt_widget=format_widget):
                if fig_cont["fig"] is None:
                    print("No figure to save, run the computation first.")
                    return
                fname = fname_widget.value.strip()
                if fname == "":
                    print("Enter a valid file name.")
                    return
                ext = fmt_widget.value
                full_filename = f"{fname}.{ext}"
                try:
                    fig_cont["fig"].savefig(full_filename)
                    print(f"Figure saved as {full_filename}")
                except Exception as e:
                    print(f"Error saving figure: {e}")

            btn_save = widgets.Button(description="Export Plot")
            btn_save.on_click(export_figure)

            torsion_blocks.append(VBox([
                phi_block,
                psi_block,
                btn_calc,
                HBox([filename_widget, format_widget, btn_save]),
                output_plot
            ]))

        display(VBox(torsion_blocks))

num_torsions_widget.observe(generate_torsion_widgets, names='value')
num_torsions_widget.disabled = True

display(num_torsions_widget, output_torsions)

BoundedIntText(value=1, description='Torsions:', disabled=True, max=10, min=1)

Output()

### 🔗 Trajectory Clustering

This section performs structural clustering of a molecular dynamics trajectory using the RMSD matrix and the k-medoids algorithm.

**Required input:**
- A trajectory file already loaded into the `traj` variable (e.g., `.xtc`, `.dcd`, etc.)
- A corresponding topology file previously loaded (e.g., `.pdb`, `.mae`)

**Output:**
- A text report showing frames per cluster, percentage, and average RMSD
- Centroid structures for each cluster in `.pdb` format
- A `.png` bar chart of the cluster distribution

Clusters are **automatically sorted by population**:  
**Cluster 1** is the most representative (e.g., 50%), followed by Cluster 2, and so on.

All results are saved inside the `Cluster_Results` folder in the selected output directory.


In [5]:
# ================================
# TRAJECTORY CLUSTERING
# ================================

import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
import os
from ipywidgets import widgets, Layout, Output, VBox, HBox
from IPython.display import display, clear_output
from tqdm.notebook import tqdm
from ipyfilechooser import FileChooser

# --- Widgets ---
cluster_output = Output()
progress_cluster = widgets.FloatProgress(min=0, max=1, value=0, description='Progress:')
n_clusters_widget = widgets.IntText(
    value=10,
    min=1,
    max=50,
    description='Number of clusters:',
    layout=Layout(width='200px')
)
run_button = widgets.Button(description="Run Clustering", button_style='info')
save_button = widgets.Button(description="Save Results", disabled=True, button_style='success')
folder_chooser = FileChooser(os.getcwd())
folder_chooser.title = '<b>Select output folder</b>'
folder_chooser.show_only_dirs = True

# Layout
cluster_box = VBox([
    widgets.HTML("<h3>Trajectory Clustering</h3>"),
    n_clusters_widget,
    run_button,
    save_button,
    folder_chooser,
    progress_cluster,
    cluster_output
])

# --- Main Functions ---
def run_clustering(b):
    global cluster_results, traj
    with cluster_output:
        clear_output()
        save_button.disabled = False

        if 'traj' not in globals():
            print("❌ ERROR: No trajectory loaded!")
            print("Please load a trajectory in previous sections")
            return

        n_clusters = n_clusters_widget.value
        progress_cluster.value = 0
        progress_cluster.description = 'Aligning...'

        try:
            traj.superpose(traj, frame=0)

            progress_cluster.description = 'Computing RMSD...'
            rmsd_matrix = np.zeros((traj.n_frames, traj.n_frames))

            for i in tqdm(range(traj.n_frames), desc="Computing RMSD"):
                rmsd_matrix[i] = md.rmsd(traj, traj, i)
                if i % 10 == 0:
                    progress_cluster.value = i / (traj.n_frames * 2)

            progress_cluster.description = 'Clustering...'
            from sklearn_extra.cluster import KMedoids

            kmedoids = KMedoids(n_clusters=n_clusters, metric='precomputed', init='k-medoids++', random_state=42)
            kmedoids.fit(rmsd_matrix)

            cluster_labels = kmedoids.labels_
            medoid_indices = kmedoids.medoid_indices_

            cluster_counts = np.bincount(cluster_labels)
            cluster_fraction = cluster_counts / len(cluster_labels) * 100

            cluster_rmsd = []
            for i in range(n_clusters):
                cluster_mask = (cluster_labels == i)
                cluster_distances = rmsd_matrix[medoid_indices[i]][cluster_mask]
                cluster_rmsd.append(np.mean(cluster_distances))

            # Sort clusters by decreasing population
            sorted_indices = np.argsort(-cluster_counts)

            sorted_labels = np.zeros_like(cluster_labels)
            for new_idx, old_idx in enumerate(sorted_indices):
                sorted_labels[cluster_labels == old_idx] = new_idx

            cluster_labels = sorted_labels
            medoid_indices = [medoid_indices[i] for i in sorted_indices]
            cluster_counts = cluster_counts[sorted_indices]
            cluster_fraction = cluster_fraction[sorted_indices]
            cluster_rmsd = [cluster_rmsd[i] for i in sorted_indices]

            cluster_results = {
                'labels': cluster_labels,
                'medoids': medoid_indices,
                'counts': cluster_counts,
                'fraction': cluster_fraction,
                'rmsd': cluster_rmsd,
                'matrix': rmsd_matrix
            }

            generate_report()

            progress_cluster.value = 1
            progress_cluster.bar_style = 'success'
            save_button.disabled = False

        except Exception as e:
            progress_cluster.bar_style = 'danger'
            print(f"❌ Clustering error: {str(e)}")


def generate_report():
    with cluster_output:
        clear_output()
        n_clusters = n_clusters_widget.value

        print("="*60)
        print(f"CLUSTERING REPORT ({n_clusters} clusters)")
        print("="*60)
        print(f"Total frames: {traj.n_frames}")
        print(f"Clustered frames: {len(cluster_results['labels'])}")
        print("="*60)
        print("Cluster details:")
        print("-"*60)
        print("Cluster | Frames | Percentage | Avg RMSD (Å) | Centroid Frame")
        print("-"*60)

        for i in range(n_clusters):
            print(f"{i+1:7d} | {cluster_results['counts'][i]:6d} | "
                  f"{cluster_results['fraction'][i]:10.2f}% | "
                  f"{cluster_results['rmsd'][i]:13.3f} | "
                  f"{cluster_results['medoids'][i]:14d}")

        plt.figure(figsize=(10, 6))
        bars = plt.bar(range(1, n_clusters+1), cluster_results['fraction'], color='skyblue')
        plt.xlabel('Cluster')
        plt.ylabel('Trajectory Percentage (%)')
        plt.title('Cluster Distribution')
        plt.xticks(range(1, n_clusters+1))

        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, height, f'{height:.1f}%', ha='center', va='bottom')

        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()

        print("="*60)
        print("✅ Clustering completed successfully!")
        print(f"Centroid frames: {cluster_results['medoids']}")


def save_results(b):
    with cluster_output:
        clear_output()
        try:
            output_dir = folder_chooser.selected_path
            if not output_dir:
                print("❌ No output folder selected.")
                return

            n_clusters = n_clusters_widget.value
            cluster_dir = os.path.join(output_dir, "Cluster_Results")
            os.makedirs(cluster_dir, exist_ok=True)

            centroids_dir = os.path.join(cluster_dir, "Centroids")
            os.makedirs(centroids_dir, exist_ok=True)

            for i, frame_idx in enumerate(cluster_results['medoids']):
                centroid = traj[frame_idx]
                centroid.save_pdb(os.path.join(centroids_dir, f"centroid_cluster_{i+1}_frame_{frame_idx}.pdb"))

            report_path = os.path.join(cluster_dir, "cluster_report.txt")
            with open(report_path, 'w') as report_file:
                report_file.write("="*60 + "\n")
                report_file.write(f"CLUSTERING REPORT ({n_clusters} clusters)\n")
                report_file.write("="*60 + "\n\n")
                report_file.write(f"Total frames: {traj.n_frames}\n")
                report_file.write(f"Clustered frames: {len(cluster_results['labels'])}\n")
                report_file.write("="*60 + "\n")
                report_file.write("Cluster details:\n")
                report_file.write("-"*60 + "\n")
                report_file.write("Cluster | Frames | Percentage | Avg RMSD (Å) | Centroid Frame\n")
                report_file.write("-"*60 + "\n")

                for i in range(n_clusters):
                    report_file.write(
                        f"{i+1:7d} | {cluster_results['counts'][i]:6d} | "
                        f"{cluster_results['fraction'][i]:10.2f}% | "
                        f"{cluster_results['rmsd'][i]:13.3f} | "
                        f"{cluster_results['medoids'][i]:14d}\n"
                    )

            plot_path = os.path.join(cluster_dir, "cluster_distribution.png")
            plt.figure(figsize=(10, 6))
            bars = plt.bar(range(1, n_clusters+1), cluster_results['fraction'], color='skyblue')
            plt.xlabel('Cluster')
            plt.ylabel('Trajectory Percentage (%)')
            plt.title('Cluster Distribution')
            plt.xticks(range(1, n_clusters+1))

            for bar in bars:
                height = bar.get_height()
                plt.text(bar.get_x() + bar.get_width()/2, height, f'{height:.1f}%', ha='center', va='bottom')

            plt.grid(axis='y', linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.savefig(plot_path, dpi=300)
            plt.close()

            print("✅ Results saved successfully!")
            print(f"Output folder: {cluster_dir}")

        except Exception as e:
            print(f"❌ Error during saving: {str(e)}")

# --- Event Binding ---
run_button.on_click(run_clustering)
save_button.on_click(save_results)

# Display the interface
display(cluster_box)
